<a href="https://colab.research.google.com/github/cathieG/Prompt_Engineering/blob/main/zero_shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run the dependencies


!pip3 install openai
!pip install evaluate
!pip install sacrebleu
!pip install tree_sitter
! git clone -q https://github.com/microsoft/CodeXGLUE.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and p

In [ ]:
"""Run this model in Python

> pip install openai
"""
import os
import json
import pandas as pd
from openai import OpenAI
import evaluate


sacrebleu = evaluate.load("sacrebleu")

# Create your PAT token by following instructions here:
# https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
# grab the token from the token.txt file
# with open("token.txt", "r") as f:
#     GITHUB_TOKEN = f.read().strip()

# Set it as an environment variable
# Insert your github token next to this line. As my github token is private, I obviously will not be showcasing it here.
# os.environ["GITHUB_TOKEN"] = GITHUB_TOKEN


# To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
client = OpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"],
)

# List of available models
model_choices = ["gpt-4o-mini" , "Codestral-2501" , "DeepSeek-R1"]
#DeepSeek is not allowed on gov owened machines

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/parser && \
# bash build.sh java


In [ ]:
# Make sure you have the OpenAI client properly set up before running this
# and that `client` and `model_choices` are defined.

code_1 = [
    '''
    Prompt: "Summarize what this function does in two sentences or less:"

    public Map<String, Integer> countWordFrequency(List<String> words) {
    Map<String, Integer> freqMap = new HashMap<>();
    for (String word : words) {
        freqMap.put(word, freqMap.getOrDefault(word, 0) + 1);
    }
    return freqMap;
}''',

    '''
    Prompt: "Identify and fix the off-by one error. Please provide only the completed funcion:"

    def sum_range(start, end):
    total = 0
    for i in range(start, end):
        total += i
    return total''',

    '''
    Prompt: "Classify the type of bug in the C++ function. Do nothing else besides this. This should be one sentence:"

    int* getArray(int size) {
    int arr[size]; // Warning: local array
    return arr; // Bug: returning pointer to local variable
}''',

    '''
    Prompt: "Complete the function using regex to validate basic email addresses. Don't explain your process, just give me the completed code:"

    def is_valid_email(email):
    # TODO: Complete using regex
    pass''',

    '''
    Prompt: "Create a‘/greet/<username>‘endpoint that returns a JSON greeting. Don't explain your process. Just give me the completed code:"

    from flask import Flask, jsonify
app = Flask(__name__)
@app.route(’/greet/<username>’)
def greet(username):
    # TODO: Return a JSON greeting
    pass''',

    '''
    Prompt: "Write the schema for a review app with users,books,andreviews. Don't explain your process, just give me the end result:"

    -- TODO: Design schema with appropriate keys and constraints
-- Tables: users(id, name), books(id, title), reviews(id, user_id, book_id, rating)''',

    '''
    Prompt: "Identify any null dereference risks in the code and summarize them briefly in a sentence or two:"

    public int getLength(String s) {
    return s.length(); // What if s is null?
}''',

    '''
    Prompt: "Improve the parser to support quoted fields. Don't explain your process, just give me the final code."

    def parse_csv_line(line):
    return line.split(’,’) # Incomplete: doesn’t handle quoted fields''',

    '''
    Prompt: "Convert the data class to a  REST API using Ktor. Just give me the final code without explaining your process:"

    data class Product(val id: Int, val name: String, val price: Double)
// TODO: Create GET and POST endpoints using Ktor''',

    '''
    Prompt: "Summarize the function in two sentences or less:"

    def reverse_words(sentence):
    return ’ ’.join(sentence.split()[::-1])''',

    '''
    Prompt: "Write a prompt that could generate the code below. No need to explain your reasoning, you can just give me the prompt:"

    # This function checks if a number is prime
def is_prime(n):
    if n <= 1:
        return False
    for i in range (2 , int ( n **0.5) +1) :
        if n % i == 0:
            return False
    return True''',

    '''
    Prompt: "Fix the bug when the input is 0. Just give me the final code without explaining anything else:"

    def factorial(n):
    result = 1
    for i in range(1, n):
        result *= i
    return result''',

    '''
    Prompt: "implement node deletion by value. Just give me the finished code, don't explain anything:"

    struct Node {
    int data;
    struct Node* next;
};
void deleteNode(struct Node** head, int key) {
    // TODO: Implement node deletion
}''',

    '''
    Prompt: "Complete the recursive function for Fibonacci. Only give me the completed code. Don't explain anything:"

    def fibonacci(n):
    # TODO: Base cases and recursive call
    pass''',

    '''
    Prompt: "Complete the class constructor. Don't explain your process, just give me the finished constructor."

    class Person:
    def __init__(self):
        # TODO: Add name, age, and optional email
        pass''',

    '''
    Prompt: "Complete the binary search implementation and return only the final code without explanation:

    public int binarySearch(int[] arr, int target) {
    int left = 0, right = arr.length- 1;
    while (left <= right) {
        int mid = (left + right) / 2;
        // TODO: Compare and adjust bounds
    }
    return -1;
}''',

    '''
    Prompt: "Resolve inconsistency between function name and logic. Don't explain what you did and why, just give me the corrected code:"

    // Supposed to return true if x is even
bool isOdd(int x) {
    return x % 2 == 0; // Logic contradicts function name
}''',

    '''
    Prompt: "Fix the bug in the function. Just give me the final code, and don't explain or write about anything else:"

    function isEven(n) {
    return n % 2; // Returns 1 or 0, not true/false
}''',

    '''
    Prompt: "Take the sumamry/comment, decompose it, then rewrite it into more logical steps. Don't explain your process, just give me your final output:"

    // Function that validates an input, calculates square, and returns result
int process(int x) {
    if (x < 0) return-1;
    return x * x;
}''',

    '''
    Prompt: "Finish the rest of the function. Just give me the final function, nothing else:"

    def calculate_average(scores):
    total = 0
    # TODO: Complete to return average
    pass''',

    '''

    Prompt: "Analyze the utility script, find any logical issues or design flaws, then refactor the code for better readability, correctness, and safety. Don't explain your process or reasoning. Just give me the refactored code with no comments:"

    # utils.py - Script to parse and summarize numeric CSV files
import csv

def read_csv(filepath):
    with open(filepath, ’r’) as f:
        return [row for row in csv.reader(f)]

def summarize_column(data, index):
    values = [float(row[index]) for row in data[1:]] # skip header
    total = sum(values)
    avg = total / len(values)
    return total, avg

def main():
    filepath = ’data.csv’
    data = read_csv(filepath)
    total, avg = summarize_column(data, 1)
    print("Total:", total)
    print("Average:", avg)

if __name__ == ’__main__’:
    main()''',

    '''

    Prompt: "Complete the following code, making sure you write in the missing code as well as cleaning up each line, removing punctuation, and making sure it counts word frequencies correctly. Don't explain your process. Just output the completed code:"

    # file_processor.py - Incomplete script for processing text files
import string

def load_file(filepath):
    with open(filepath, ’r’) as f:
        return f.readlines()

def clean_line(line):
    # TODO: Remove punctuation and make lowercase
    pass

def count_words(lines):
    word_counts = {}
    for line in lines:
        clean = clean_line(line)
        for word in clean.split():
            word_counts[word] = word_counts.get(word, 0) + 1
    return word_counts

def main():
    filepath = ’input.txt’
    lines = load_file(filepath)
    counts = count_words(lines)
    for word, count in sorted(counts.items()):
        print(f"{word}: {count}")

if __name__ == ’__main__’:
    main()'''
]



In [ ]:
# print(len(code_test))

In [ ]:
# zero_shot_prompts = ["Summarize what this function does in two sentences or less:",
#                      "Identify and fix the off-by one error. Please provide only the completed funcion",
#                      "Classify the type of bug in the C++ function. Do nothing else besides this. This should be one sentence",
#                      "Complete the function using regex to validate basic email addresses. Don't explain your process, just give me the completed code.",
#                      "Create a‘/greet/<username>‘endpoint that returns a JSON greeting. Don't explain your process. Just give me the completed code.",
#                      "Write the schema for a review app with users,books,andreviews. Don't explain your process, just give me the end result.",
#                      "Identify any null dereference risks in the code and summarize them briefly in a sentence or two.",
#                      "Improve the parser to support quoted fields. Don't explain your process, just give me the final code.",
#                      "Convert the data class to a  REST API using Ktor. Just give me the final code without explaining your process.",
#                      "Summarize the function in two sentences or less:",
#                      "Write a prompt that could generate the code below. No need to explain your reasoning, you can just give me the prompt.",
#                      "Fix the bug when the input is 0. Just give me the final code without explaining anything else.",
#                      "implement node deletion by value. Just give me the finished code, don't explain anything.",
#                      "Complete the recursive function for Fibonacci. Only give me the completed code. Don't explain anything.",
#                      "Complete the class constructor. Don't explain your process, just give me the finished constructor.",
#                      "Complete the binary search implementation and return only the final code without explanation.",
#                      "Resolve inconsistency between function name and logic. Don't explain what you did and why, just give me the corrected code.",
#                      "Fix the bug in the function. Just give me the final code, and don't explain or write about anything else.",
#                      "Take the sumamry/comment, decompose it, then rewrite it into more logical steps. Don't explain your process, just give me your final output.",
#                      "Finish the rest of the function. Just give me the final function, nothing else:",
#                      "Analyze the utility script, find any logical issues or design flaws, then refactor the code for better readability, correctness, and safety. Don't explain your process or reasoning. Just give me the refactored code with no comments.",
#                      "Complete the following code, making sure you write in the missing code as well as cleaning up each line, removing punctuation, and making sure it counts word frequencies correctly. Don't explain your process. Just output the completed code."]


In [ ]:
# Define function that produces the output, then put that output into two seperate lists

def prompt_model_zero(code):
  Claude = []
  GPT = []

  i = 0

  for code_snippets in code:

    print("This is the code_snippet: ")
    print(code_snippets)

    response_Claude = client.chat.completions.create(
        model=model_choices[1],
        messages=[
            {"role": "user", "content":code_snippets}
        ],
        max_tokens=1024,
        temperature=0.3,
    )

    response_GPT = client.chat.completions.create(
        model=model_choices[0],
        messages=[
            {"role": "user", "content": code_snippets}
        ],
        max_tokens=1024,
        temperature=0.3,
    )

    output_Claude = response_Claude.choices[0].message.content
    Claude.append({
        "index": i,
        "prompt": zero_shot_prompts[i],
        "code_snippet": code_snippets,
        "response": output_Claude
    })

    output_GPT = response_GPT.choices[0].message.content
    GPT.append({
        "index": i,
        "prompt": zero_shot_prompts[i],
        "code_snippet": code_snippets,
        "response": output_GPT
    })

    i += 1
    print(f"Response for Claude, Code Snippet {i}:\n{response_Claude.choices[0].message.content}\n{'-'*60}\n")
    print()
    print(f"Response for GPT, Code Snippet {i}:\n{response_GPT.choices[0].message.content}\n{'-'*60}\n")
    print()

  return Claude, GPT

In [ ]:
# Run the Code

pred, ref = prompt_model_zero(code_1)

# print(pred)
# print(ref)
# print(len(pred))
# print(len(ref))

This is the code_snippet: 

    Prompt: "Improve the parser to support quoted fields. Don't explain your process, just give me the final code."

    def parse_csv_line(line):
    return line.split(’,’) # Incomplete: doesn’t handle quoted fields


BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': True, 'detected': True}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}

In [ ]:
# Put them into predictions and references

predictions = []
references = []
for i in range(22):
  predictions.append(pred[i]['response'])
  references.append(ref[i]['response'])

In [ ]:
# Get the evaluation scores for each output

def Calc_Metrics_Zero(prediction, reference):
  scores = []
  selected_indices = {0, 2, 6, 9, 10, 18}
  for i, (pred, ref) in enumerate(zip(predictions, references)):
    print(i)
    print(pred)
    print(ref)
    print()
    if i in selected_indices:

      result = sacrebleu.compute(predictions=[pred], references=[[ref]])
      print("Bleu-4 Score: "+ str(result['score']))
      print()
      scores.append("Bleu-4 Score: " + str(round(result["score"], 5)))

    else:
      # print("Exact Match: "+ str(pred == ref))
      # print()
      scores.append("Match: " +str(pred == ref))

  return scores

In [ ]:
New_Scores = Calc_Metrics_Zero(predictions, references)
# print(New_Scores)

0
This function takes a list of words and returns a map where the keys are the words from the list and the values are the frequencies of those words. It iterates through the list, updating the count for each word in the map.
The function `countWordFrequency` takes a list of words as input and returns a map that counts the frequency of each word in the list. It uses a HashMap to store the words as keys and their corresponding counts as values, updating the count for each word as it iterates through the list.

Bleu-4 Score: 30.65076310801197

1
```python
def sum_range(start, end):
    total = 0
    for i in range(start, end + 1):
        total += i
    return total
```
```python
def sum_range(start, end):
    total = 0
    for i in range(start, end + 1):
        total += i
    return total
```

2
Returning pointer to local variable
The bug is returning a pointer to a local variable, which leads to undefined behavior.

Bleu-4 Score: 2.7409299216719143

3
```python
import re

def is_valid_

In [ ]:
scores = New_Scores

In [ ]:



# with open('Codestral.json', 'w') as json_file:
#     json.dump(predictions, json_file, indent=4)

# with open('GPT.json', 'w') as json_file:
#   json.dump(references, json_file, indent=4)

# with open('Scores.json', 'w') as json_file:
#   json.dump(scores, json_file, indent=4)

In [ ]:
# predictions = []
# references = []
# scores = []

In [ ]:
# with open('Codestral.json', 'r') as json_file:
#     predictions = json.load(json_file)

# with open('GPT.json', 'r') as json_file:
#   references = json.load(json_file)
# with open('Scores.json', 'r') as json_file:
#   scores = json.load(json_file)

FileNotFoundError: [Errno 2] No such file or directory: 'Codestral.json'

In [ ]:
# Format the data into

data = []

for i, prompt in enumerate(code_1):
    for model_name in ["Codestral-2501", "GPT-4o-mini"]:
      if model_name == "Codestral-2501":
        output = predictions[i]
      elif model_name == "GPT-4o-mini":
        output= references[i] # Your function
      score = scores[i]  # Your metric
      analysis = "Generic text"  # Optional
      data.append({
            "prompt_id": i + 1,
            "parameters": "max_tokens=1024,\n temperature = 0.3",
            "prompt_text": prompt,
            "model_name": model_name,
            "output_text": output,
            "score": score,
            # "analysis": analysis
        })

df = pd.DataFrame(data)


In [ ]:
df.to_csv('zero_shot_final.csv', index=False)